In [33]:
import pandas as pd
from numpy import array
import datetime
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [34]:
df = pd.read_csv('/Users/stewarta/repos/iot-htsensor/data/temp_heat.csv')
df.head()

,datetime,Temp1,Temp2,Temp3,Temp4,Humi1,Humi2,Humi3,Humi4
0,2013-04-14 07:00:00,10.229167,19.600000,22.975000,20.200000,76.000000,42.000000,34.000000,40.000000
1,2013-04-14 08:00:00,11.390196,19.590196,23.028431,20.199020,75.460784,42.401961,34.392157,40.725490
2,2013-04-14 09:00:00,13.222772,19.518812,23.630693,20.179208,73.851485,43.000000,34.138614,41.000000
3,2013-04-14 10:00:00,14.580198,19.624752,23.916832,20.383168,73.049505,43.653465,34.415842,41.881188
4,2013-04-14 11:00:00,16.541748,19.560194,24.039806,20.441748,72.087379,45.203883,35.436893,42.970874


In [35]:
# ensure timestamp is date time
df['datetime'] = pd.to_datetime(df.datetime, errors='coerce')

In [36]:
# set the timestamp as index
df.set_index('datetime', inplace=True)

In [37]:
# create a subset for analysis
sensor = 'Temp1'
filter = (df.index.year == 2013) & (df.index.month == 4) 
y = np.array(df[filter][sensor])
raw_seq = y[:20]

In [38]:
# univariate data preparation

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [39]:
# define input sequence
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10.22916667 11.39019608 13.22277228] 14.580198019801974
[11.39019608 13.22277228 14.58019802] 16.541747572815535
[13.22277228 14.58019802 16.54174757] 17.90396039603961
[14.58019802 16.54174757 17.9039604 ] 17.224999999999998
[16.54174757 17.9039604  17.225     ] 17.456000000000007
[17.9039604 17.225     17.456    ] 17.555445544554445
[17.225      17.456      17.55544554] 17.642999999999983
[17.456      17.55544554 17.643     ] 17.194059405940585
[17.55544554 17.643      17.19405941] 16.41372549019606
[17.643      17.19405941 16.41372549] 15.691176470588221
[17.19405941 16.41372549 15.69117647] 15.318627450980385
[16.41372549 15.69117647 15.31862745] 15.093000000000007
[15.69117647 15.31862745 15.093     ] 14.745098039215668
[15.31862745 15.093      14.74509804] 14.467000000000018
[15.093      14.74509804 14.467     ] 14.272277227722785
[14.74509804 14.467      14.27227723] 14.012621359223328
[14.467      14.27227723 14.01262136] 13.581372549019632


In [49]:
xtrain = X[0:len(X)-1]
xtest = X[-1]

ytrain = y[0:len(X)-1]
ytest = y[-1]

print('=======  x train ========')
print(xtrain)
print('=======  x test ========')
print(xtest)


print('========= y train ======')
print(ytrain)
print('========= y test ======')
print(ytest)

=======  x train ========
[[10.22916667 11.39019608 13.22277228]
 [11.39019608 13.22277228 14.58019802]
 [13.22277228 14.58019802 16.54174757]
 [14.58019802 16.54174757 17.9039604 ]
 [16.54174757 17.9039604  17.225     ]
 [17.9039604  17.225      17.456     ]
 [17.225      17.456      17.55544554]
 [17.456      17.55544554 17.643     ]
 [17.55544554 17.643      17.19405941]
 [17.643      17.19405941 16.41372549]
 [17.19405941 16.41372549 15.69117647]
 [16.41372549 15.69117647 15.31862745]
 [15.69117647 15.31862745 15.093     ]
 [15.31862745 15.093      14.74509804]
 [15.093      14.74509804 14.467     ]
 [14.74509804 14.467      14.27227723]]
=======  x test ========
[14.467      14.27227723 14.01262136]
========= y train ======
[14.58019802 16.54174757 17.9039604  17.225      17.456      17.55544554
 17.643      17.19405941 16.41372549 15.69117647 15.31862745 15.093
 14.74509804 14.467      14.27227723 14.01262136]
========= y test ======
13.581372549019632


In [54]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(xtrain, ytrain, epochs=2000, verbose=0)
# demonstrate prediction
x_input = xtest
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[13.91739]]
